In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
import zarr
import glob
import s3fs
import json
import fsspec

In [2]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client

In [3]:
cluster = KubeCluster()
cluster.adapt(minimum=2, maximum=20, interval='2s', wait_count=3)
client = Client(cluster)
cluster

In [28]:
%%time
import fsspec
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)

CPU times: user 7.72 s, sys: 299 ms, total: 8.02 s
Wall time: 45 s


In [75]:
#ds_sst

In [76]:
lat_slice = slice(0, 45)
lon_slice = slice(0, 45)
ds_subset = ds_sst.sel(lat=lat_slice, lon=lon_slice, time=slice('2003-01-01', '2005-01-01'))
ds_subset

,Array,Chunk
Bytes,59.32 GB,129.53 MB
Shape,"(732, 4501, 4501)","(5, 1799, 3600)"
Count,143123 Tasks,1332 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,59.32 GB,129.53 MB
Shape,"(732, 4501, 4501)","(5, 1799, 3600)"
Count,143123 Tasks,1332 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [77]:
ds_subset.analysed_sst.encoding = {}
ds_subset.analysis_error.encoding = {}
ds_subset.mask.encoding = {}
ds_subset.sea_ice_fraction.encoding = {}

In [82]:
%%time
s3 = s3fs.S3FileSystem(
    anon=False,
    key='xxx',    
    secret='xxx',
    client_kwargs=dict(region_name='us-west-2')
)

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 35.8 µs


In [84]:
%%time

chunks = {'time': 60, 'lat': 1799, 'lon': 3600}

ds_rechunked = ds_subset.chunk(chunks)
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'writing to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'w'}
ds_rechunked.to_zarr(s3store, **args)

writing to mur-sst/zarr-60_1799_3600
CPU times: user 2min 8s, sys: 3.17 s, total: 2min 11s
Wall time: 11min 21s


In [85]:
%%time

file_location = f's3://mur-sst/zarr-{chunk_string}'
first_chunks = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
first_chunks

CPU times: user 689 ms, sys: 27.4 ms, total: 716 ms
Wall time: 2.67 s


<xarray.Dataset>
Dimensions:           (lat: 4501, lon: 4501, time: 732)
Coordinates:
  * lat               (lat) float32 0.0 0.01 0.02 0.03 ... 44.98 44.99 45.0
  * lon               (lon) float32 0.0 0.01 0.02 0.03 ... 44.98 44.99 45.0
  * time              (time) datetime64[ns] 2003-01-01T09:00:00 ... 2005-01-01T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(60, 1799, 3600), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [87]:
%%time

chunks = {'time': 180, 'lat': 1023, 'lon': 2047}

ds_rechunked = ds_subset.chunk(chunks)
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'writing to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'w'}
ds_rechunked.to_zarr(s3store, **args)

writing to mur-sst/zarr-180_1023_2047
CPU times: user 2min 49s, sys: 4.16 s, total: 2min 54s
Wall time: 14min 16s


In [88]:
file_location = f's3://mur-sst/zarr-{chunk_string}'
second_chunks = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
second_chunks

<xarray.Dataset>
Dimensions:           (lat: 4501, lon: 4501, time: 732)
Coordinates:
  * lat               (lat) float32 0.0 0.01 0.02 0.03 ... 44.98 44.99 45.0
  * lon               (lon) float32 0.0 0.01 0.02 0.03 ... 44.98 44.99 45.0
  * time              (time) datetime64[ns] 2003-01-01T09:00:00 ... 2005-01-01T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(180, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(180, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(180, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(180, 1023, 2047), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Resolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_name:               JPL MUR SST project
    creator_url:                http://mur.jpl.nasa.gov
    date_created:               20200124T010755Z
    easternmost_longitude:      180.0
    file_quality_level:         3
    gds_version_id:             2.0
    geospatial_lat_resolution:  0.009999999776482582
    geospatial_lat_units:       degrees north
    geospatial_lon_resolution:  0.009999999776482582
    geospatial_lon_units:       degrees east
    history:                    created at nominal 4-day latency; replaced nr...
    id:                         MUR-JPL-L4-GLOB-v04.1
    institution:                Jet Propulsion Laboratory
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    These data are available free of charge under...
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.1
    northernmost_latitude:      90.0
    platform:                   Terra, Aqua, GCOM-W, MetOp-A, MetOp-B, Buoys/...
    processing_level:           L4
    product_version:            04.1
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    sensor:                     MODIS, AMSR2, AVHRR, in-situ
    source:                     MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRM...
    southernmost_latitude:      -90.0
    spatial_resolution:         0.01 degrees
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    start_time:                 20200116T090000Z
    stop_time:                  20200116T090000Z
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20200116T210000Z
    time_coverage_start:        20200115T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [ ]:
%%time

chunks = {'time': 379, 'lat': 439, 'lon': 360}

ds_rechunked = ds_subset.chunk(chunks)
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'writing to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'w'}
ds_rechunked.to_zarr(s3store, **args)

writing to mur-sst/zarr-379_439_360


distributed.scheduler - ERROR - 
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/scheduler.py", line 1977, in update_graph
    resources=resources,
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/diagnostics/websocket.py", line 33, in update_graph
    self.socket.send("update_graph", {"client": client})
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/dashboard/scheduler.py", line 350, in send
    self.write_message(data)
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/tornado/websocket.py", line 339, in write_message
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError


In [ ]:
file_location = f's3://mur-sst/zarr-{chunk_string}'
third_chunks = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
third_chunks

This is only 0.3% of the data, so concerned about overall write time. If it scales linearly we are looking at up to 4 days.

## Performance Tests

In [ ]:
%%time
first_chunks.sel(time=slice('2004-01-01','2004-02-01')).mean(dim='time')
#first_chunks.sel(time='2004-10-01',lat=slice(20,30)).mean(dim='time')

In [ ]:
%%time
second_chunks.sel(time=slice('2004-01-01','2004-02-01')).mean(dim='time')

In [ ]:
%%time
third_chunks.sel(time=slice('2004-01-01','2004-02-01')).mean(dim='time')

## Try appending

In [ ]:
ds_subset = ds_sst.sel(lat=lat_slice, lon=lon_slice, time=slice('2005-01-01', '2006-01-01'))
ds_subset.analysed_sst.encoding = {}
ds_subset.analysis_error.encoding = {}
ds_subset.mask.encoding = {}
ds_subset.sea_ice_fraction.encoding = {}
ds_subset

In [ ]:
%%time

chunks = {'time': 60, 'lat': 1799, 'lon': 3600}

ds_rechunked = ds_subset.chunk(chunks)
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f'mur-sst/zarr-{chunk_string}'
print(f'appending to {rechunk_location}')

s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
args = {'consolidated': True, 'mode': 'a', 'append_dim': 'time'}
ds_rechunked.to_zarr(s3store, **args)